# 1. Discretize the state space

In [16]:
import numpy as np
from LSTM import LSTM
from System import Plant
from Policy import Estimator
import warnings
from tqdm import tqdm
warnings.filterwarnings('ignore')
np.random.seed(0)

resolution = 0.05

c_dis = np.arange(start=0, stop=7.01, step=resolution)
p_dis = np.arange(start=0, stop=17.01, step=resolution)
l_dis = np.arange(start=0, stop=10.01, step=resolution)
u_dis = np.arange(start=-5, stop=-4, step=1)

def close_idx(list, target):
    return np.argmin(np.abs(list - target))

def get_idx(p_idx, c_idx, l_idx):
    return (p_idx * c_dis.shape[0] * l_dis.shape[0] + 
            c_idx * l_dis.shape[0]                  +
            l_idx)
def get_pcl(idx):
    p_idx = idx // (c_dis.shape[0] * l_dis.shape[0])
    c_idx = (idx % (c_dis.shape[0] * l_dis.shape[0])) // l_dis.shape[0]
    l_idx = idx % l_dis.shape[0]
    return [p_dis[p_idx], c_dis[c_idx], l_dis[l_idx]]

n_sample = 100

n = c_dis.shape[0] * p_dis.shape[0] * l_dis.shape[0]

# 2. Generate transition matrix

In [29]:
P_trans = {}
P_trans[0,0,0] = 1

plant = Plant(dt=1/60)
estimator = Estimator()
std = 0.1 # variation of estimated load

for u_idx, u in enumerate(u_dis):
    for p_idx, p in tqdm(enumerate(p_dis)):
        for c_idx, c in enumerate(c_dis):
            for l_idx, l in enumerate(l_dis):
                real_loads = np.random.normal(loc=estimator.estimate(l), scale=std, size=n_sample)
                for real_l in real_loads:
                    state_idx = get_idx(p_idx, c_idx, l_idx)
                    plant.reset(c, p, l)
                    plant.step(u, real_l)
                    hit_state_idx = get_idx(close_idx(p_dis, plant.p),
                                            close_idx(c_dis, plant.battery.c),
                                            close_idx(l_dis, plant.l))
                    if P_trans.__contains__((u_idx, state_idx, hit_state_idx)):
                        P_trans[u_idx, state_idx, hit_state_idx] += (1/n_sample)
                    else:
                        P_trans[u_idx, state_idx, hit_state_idx] = (1/n_sample)



3it [04:05, 81.87s/it]


KeyboardInterrupt: 

In [15]:
import matplotlib.pyplot as plt
print(get_pcl(np.argmax(P_trans[0,0,:])))


[0.0, 0.0, 0.0]


# 3. Dynamic programming

In [51]:
# Initialization of time horizon and cost at time T
T = 1000
J = np.array([get_pcl(i)[0] for i in range(n)]).reshape(-1,1)

for t in tqdm(range(1000, -1, -1)):
    J_hat = np.zeros(shape=[n, 1])
    for i in range(n):
        J_hat[i]
    